In [41]:
import sys
import time
import requests
from datetime import datetime, timedelta
import json
import fasttext
import re
from bs4 import BeautifulSoup
import lxml
import numpy as np
import os
import unidecode 
import pandas as pd


EARLIEST_DATE = datetime(2019, 12, 19)
DIRNAME = 'state_analysis/'
# OUTPUT_DIRNAME = 'state_by_date/'

# GET ALL STATE DATA IN ONE PLACE

In [42]:
reddit_url = 'https://www.reddit.com/r/LocationReddits/wiki/faq/northamerica'
page = requests.get(reddit_url, headers = {'User-Agent': 'Mozilla/5.0'})
html = BeautifulSoup(page.text, "lxml")
states = html.find_all("h5")[13:]
state_sub_map = {}
for state in states:
    if state.text == 'California':
        state_sub_map[state.text] = ['california']
        continue
    links = state.findNext().find_all('a')
    links = [link['href'][3:] for link in links]
    state_sub_map[state.text] = links
cali = html.find_all("h6")
for part in cali:
    links = part.findNext().find_all('a')
    links = [link['href'][3:] for link in links]
    state_sub_map['California'].extend(links)

In [52]:
def preproc(line):
    line = unidecode.unidecode(line)
    line = re.sub(r"&gt;.*?(\n|\s\s|\Z)", "", line)
    line = re.sub(r'\s\(\s', " ", line)
    line = re.sub(r"\n", " ", line)
    line = re.sub("\su/[a-zA-Z0-9-_]*\s", " USERNAME ", line)
    line = re.sub("\sr/[a-zA-Z0-9-_]*\s", " REDDIT ", line)    
    line = re.sub("\s{2,}", " ", line)
    print(line)
    return line

In [44]:
test = "Editorial cartoon by Bill Bramhall, May 11, 2020.\n\n[Link](https://www.nydailynews.com/opinion/ny-bramhall-editorial-cartoons-2020-20200107-7ctvclbkyng5nl3ocyeet2cax4-photogallery.html)\n\nThe coronacoaster continues  🎢 \n\n[GoLocalProv](https://www.golocalprov.com/news/19-new-ri-coronavirus-deaths-highest-in-5-days-watch-saturday-1-pm-briefing)link from 5/9/2020\n\n&gt;On Saturday, Rhode Island Governor lifted the stay at home order in effect in Rhode Island. \n\n**EDITOR'S NOTE: Raimondo does not allow Rhode Islanders to hear reporters' questions. Nearly all governors across the country and the White House allow reporters to ask questions directly and for the public to hear the questions and the responses.**\n\nFrom [WPRI](https://www.wpri.com/health/coronavirus/may-11-ri-coronavirus-update/) \n\nTakeaways: \n\n&gt;There were 176 new positive cases reported Monday, about **8%** of the 2,119 total test results returned.\n\n&gt;”I only want to reopen this economy once,” Raimondo said. ” I don’t want this to be fits and starts.”\n\nYeah, ok Gina. This isn’t about what *you* want. \n\n&gt;**Reporters were allowed back in the room to ask questions for the first time in nearly two months on Monday.** The location of the briefing was moved to the Veterans Memorial Auditorium, which sits next to the R.I. State House, to allow for more spacing."
# print(test)
output = preproc(test)
print(output)

Editorial cartoon by Bill Bramhall, May 11, 2020. [Link](https://www.nydailynews.com/opinion/ny-bramhall-editorial-cartoons-2020-20200107-7ctvclbkyng5nl3ocyeet2cax4-photogallery.html) The coronacoaster continues [GoLocalProv](https://www.golocalprov.com/news/19-new-ri-coronavirus-deaths-highest-in-5-days-watch-saturday-1-pm-briefing)link from 5/9/2020 **EDITOR'S NOTE: Raimondo does not allow Rhode Islanders to hear reporters' questions. Nearly all governors across the country and the White House allow reporters to ask questions directly and for the public to hear the questions and the responses.** From [WPRI](https://www.wpri.com/health/coronavirus/may-11-ri-coronavirus-update/) Takeaways: Yeah, ok Gina. This isn't about what *you* want. 


In [48]:
state_sub_map

{'Alabama': ['alabama',
  'Anniston',
  'ArabAlabama',
  'auburn',
  'baldwincountyal',
  'birmingham',
  'blountsville',
  'Calera',
  'CalhounCounty',
  'Cullman',
  'dothan',
  'DothanAL',
  'fairhopeAlabama',
  'FayetteCounty',
  'florenceal',
  'gadsden',
  'gordo',
  'greenvilleal',
  'Hartselle',
  'cityofhoover',
  'HooverAL',
  'huntsvillealabama',
  'JacksonvilleAL',
  'Madisonalabama',
  'marshallcountyal',
  'mobileal',
  'montgomery',
  'mountainbrook',
  'oba',
  'OxfordAL',
  'prattville',
  'Ragland',
  'selma',
  'shoals',
  '251',
  'SummerdaleAL',
  'sylacauga',
  'talladega',
  'ThomasvilleAL',
  'Trussville',
  'tuscaloosa',
  'vestavia',
  'vestaviahills'],
 'Alaska': ['alaska',
  'AleutianIslands',
  'Anchorage',
  'BarrowAK',
  'BethelAK',
  'Cordova',
  'fairbanks',
  'homerAK',
  'juneau',
  'Kenai',
  'ketchikan',
  'kodiak',
  'diomede',
  'MatSu',
  'nome',
  'PalmerAK',
  'Seward',
  'Sitka',
  'skagway',
  'spenardak',
  'sterlingalaska',
  'Talkeetna',
 

In [53]:
for state in state_sub_map:
    state_data = []
    for sub in state_sub_map[state]:
        print(sub)
        if sub[-1] == "/":
            sub = sub[:-1]
        with open(DIRNAME + f'r.{sub}.submission.json', 'r') as file:
            state_data.extend([json.loads(preproc(line)) for line in file.readlines() if line != ''])
        with open(DIRNAME + f'r.{sub}.comment.json', 'r') as file:
            state_data.extend([json.loads(preproc(line)) for line in file.readlines() if line != ''])
#     print(state_data)
    with open(DIRNAME + f"{state}.all_data.json", "w", encoding="utf-8") as file:
        for line in state_data:
            json.dump(line, file)
            file.write("\n")

alabama
{"author": "Margetta", "created_utc": 1589394730, "id": "gj4pfw", "permalink": "/r/Alabama/comments/gj4pfw/alabama_companies_working_to_combat_covid19/", "selftext": "While the world continues its fight against novel coronavirus, researchers and private industry have banded together in order to slow the spread of the virus and develop new tests and treatments for COVID-19. Now, more than ever, it is evident that collaboration is necessary to overcome current challenges and nowhere is as uniquely qualified to help foster these collaborations than Huntsville\u2019s HudsonAlpha Institute for Biotechnology. LINK ", "title": "Alabama companies working to combat COVID-19"} 
{"author": "MushinZero", "created_utc": 1589141375, "id": "gh8j9l", "permalink": "/r/Alabama/comments/gh8j9l/terrified_for_my_mother_and_grandmother_why_is/", "selftext": "Currently my grandmother is staying with my mother while they wait out this Covid virus. Now that the state is starting to re-open, my mother's

JSONDecodeError: Unterminated string starting at: line 1 column 32 (char 31)

# DEMOCRAT VS REPUBLICAN STATE

# CULTURAL DIVISIONS
Sort into regions by date

In [56]:
def get_dates(latest_date):
    # Get all week increments from earliest date until present
    dates = []
    epochs = []
    curr = EARLIEST_DATE
    while curr < latest_date:
        dates.append(curr.strftime("%m/%d/%Y"))
        epochs.append(int(curr.timestamp()))
        curr += timedelta(weeks=1)
    dates.append(latest_date.strftime("%m/%d/%Y"))
    epochs.append(int(latest_date.timestamp()))
    return dates, epochs

In [57]:
def get_subreddit_data(subreddit):
    with open(f'{DIRNAME}{subreddit}.all_data.json', 'r') as file:
        data = file.readlines()
        return [json.loads(line) for line in data] 

In [61]:
def get_region_data():
    state_to_region = pd.read_csv('state_to_region.csv')
    dates, epochs = get_dates(datetime.utcnow())
    for i in range(1, 5):
        x = state_to_region.dropna().groupby(by=f'Region {i}')['State'].apply(list)
        print(i)
        for key in x.keys():
            print(key)
            region_data = []
            for state in x[key]:
                state_data = get_subreddit_data(state)
                print(len(state_data))
                return
                region_data.extend(state_data)
            times = []
            for ele in region_data:
                times.append(ele['created_utc'])
            time_buckets = np.digitize(times, epochs)
            region_data = np.array(region_data)
            for j in range(1, max(time_buckets)+1):
                mask = np.where(time_buckets==j)
                all_json = region_data[mask]
                text_data = [ele['body'] for ele in all_json if 'body' in ele.keys()]
                text_data.extend([ele['selftext'] for ele in all_json if 'selftext' in ele.keys()])
                with open(f"Region{i}/Week{j}.{key}.txt", "w", encoding="UTF-8") as region_date_file:
                    for line in text_data:
                        region_date_file.write(line)
                        region_date_file.write("\n")
                    print(f"Week{j}.{key} COMPLETE!")
#             with open(f"Region{i+1}/{key}.txt", "w") as file:
#                 print(x[key])

In [62]:
get_region_data()

1
Midwest
11911


In [157]:
ala = get_subreddit_data('Alabama')
\u30c4
Week 11
#TODO: GET ALL DATA PER REGION

In [169]:
dates, epochs = get_dates(datetime.utcnow())
times = []
for uttime in ala:
    times.append(uttime['created_utc'])
p = np.digitize(times, epochs)
print(len(p))
# print(times[:5])
# print(ala[:5])

4107


In [217]:
for i in range(1, max(p)+1):
    mask = np.where(p==i)
    ala = np.array(ala)
    all_json = ala[mask]
    text_data = [ele['body'] for ele in all_json if 'body' in ele.keys()]
    text_data.extend([ele['selftext'] for ele in all_json if 'selftext' in ele.keys()])
    

0
0
0
0
0
0
0
0
0
0
2
2
6
6
4
4


In [211]:
np.where(p==6)

(array([1089, 3947], dtype=int64),)

In [197]:
print(max(p))`
epochs[20] < ala[0]['created_utc']

21


True

In [154]:

#             file.write()
#             file.write("\n")


0
['Illinois', 'Indiana', 'Iowa', 'Michigan', 'Minnesota', 'Missouri', 'Ohio', 'Wisconsin']
['Arizona', 'Arkansas', 'Colorado', 'Idaho', 'Kansas', 'Montana', 'Nebraska', 'New Mexico', 'North Dakota', 'Oklahoma', 'South Dakota', 'Texas', 'Utah', 'Wyoming']
['Connecticut', 'Delaware', 'Maine', 'Maryland', 'Massachusetts', 'New Hampshire', 'New Jersey', 'New York', 'Pennsylvania', 'Rhode Island', 'Vermont']
['Alabama', 'Florida', 'Georgia', 'Kentucky', 'Louisiana', 'Mississippi', 'North Carolina', 'South Carolina', 'Tennessee', 'Virginia', 'West Virginia']
['Alaska', 'California', 'Hawaii', 'Nevada', 'Oregon', 'Washington']
1
['Illinois', 'Indiana', 'Iowa', 'Kansas', 'Michigan', 'Minnesota', 'Missouri', 'Nebraska', 'North Dakota', 'Ohio', 'South Dakota', 'Wisconsin']
['Connecticut', 'Delaware', 'Maine', 'Massachusetts', 'New Hampshire', 'New Jersey', 'New York', 'Pennsylvania', 'Rhode Island', 'Vermont']
['Alabama', 'Arkansas', 'Florida', 'Georgia', 'Kentucky', 'Louisiana', 'Maryland', 'M

In [6]:
# TODO FOR DISTRICT BY DISTRICT ANALYSIS
# Get rid of state full name
# get rid of state abbreviation
# Add space between county names
state_sub_map['Alabama']

['alabama',
 'Anniston',
 'ArabAlabama',
 'auburn',
 'baldwincountyal',
 'birmingham',
 'blountsville',
 'Calera',
 'CalhounCounty',
 'Cullman',
 'dothan',
 'DothanAL',
 'fairhopeAlabama',
 'FayetteCounty',
 'florenceal',
 'gadsden',
 'gordo',
 'greenvilleal',
 'Hartselle',
 'cityofhoover',
 'HooverAL',
 'huntsvillealabama',
 'JacksonvilleAL',
 'Madisonalabama',
 'marshallcountyal',
 'mobileal',
 'montgomery',
 'mountainbrook',
 'oba',
 'OxfordAL',
 'prattville',
 'Ragland',
 'selma',
 'shoals',
 '251',
 'SummerdaleAL',
 'sylacauga',
 'talladega',
 'ThomasvilleAL',
 'Trussville',
 'tuscaloosa',
 'vestavia',
 'vestaviahills']

# ORDER ALL POSTS BY DATE 